In [1]:
import numpy as np
import pandas as pd

In [2]:
df_trips = pd.read_csv('trips_1.5M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,1,2015-06-03 10:41:50,2015-06-03 11:08:33,1,2.80,-73.952888,40.776814,1,N,-73.979034,40.756611,1,17.5,0.0,0.5,4.57,0.0,22.87
1,2,2015-02-14 19:38:55,2015-02-14 20:00:59,1,2.79,-73.995827,40.725353,1,N,-73.984314,40.757389,2,15.0,0.0,0.5,0.00,0.0,15.80
2,2,2015-04-21 15:21:04,2015-04-21 15:33:38,1,1.75,-73.951920,40.769421,1,N,-73.956421,40.787140,2,9.5,0.0,0.5,0.00,0.0,10.30
3,2,2015-01-29 18:58:54,2015-01-29 19:04:46,3,0.83,-74.009026,40.715710,1,N,-74.003418,40.723152,1,5.5,1.0,0.5,1.46,0.0,8.76
4,1,2015-06-05 10:47:29,2015-06-05 10:58:00,3,0.80,-73.985909,40.756176,1,N,-73.988358,40.747398,2,7.0,0.0,0.5,0.00,0.0,7.80


In [3]:
df_trips = pd.read_csv('trips2020_1.5M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,01/01/2020 02:50:43 AM,01/01/2020 02:53:10 AM,1.0,0.80,1.0,N,141,229,1.0,4.5,3.0,0.5,1.00,0.0,0.3,9.30,2.5
1,1.0,02/24/2020 10:38:12 AM,02/24/2020 11:09:11 AM,1.0,5.60,1.0,N,100,87,2.0,23.5,2.5,0.5,0.00,0.0,0.3,26.80,2.5
2,2.0,01/18/2020 11:52:03 PM,01/19/2020 11:25:26 PM,1.0,5.04,1.0,N,230,7,1.0,18.0,0.5,0.5,4.36,0.0,0.3,26.16,2.5
3,2.0,02/02/2020 11:14:26 PM,02/02/2020 11:26:56 PM,1.0,1.88,1.0,N,255,17,1.0,10.0,0.5,0.5,2.26,0.0,0.3,13.56,0.0
4,2.0,03/11/2020 07:03:20 PM,03/11/2020 07:18:51 PM,2.0,2.32,1.0,N,264,264,1.0,11.5,1.0,0.5,1.00,0.0,0.3,16.80,2.5


In [4]:
df_trips = pd.read_csv('trips2020_3M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,01/03/2020 07:30:30 PM,01/03/2020 07:49:54 PM,0.0,2.30,1.0,Y,232,234,1.0,13.0,3.5,0.5,3.45,0.00,0.3,20.75,2.5
1,2.0,01/17/2020 05:55:47 PM,01/17/2020 07:02:12 PM,1.0,18.36,2.0,N,132,100,1.0,52.0,4.5,0.5,13.18,6.12,0.3,79.10,2.5
2,2.0,02/27/2020 10:27:00 PM,02/27/2020 10:35:02 PM,2.0,0.90,1.0,N,230,162,2.0,6.5,0.5,0.5,0.00,0.00,0.3,10.30,2.5
3,1.0,02/06/2020 12:46:47 PM,02/06/2020 01:02:42 PM,2.0,1.80,1.0,N,237,239,1.0,11.5,2.5,0.5,3.70,0.00,0.3,18.50,2.5
4,2.0,01/07/2020 11:00:17 PM,01/07/2020 11:08:49 PM,5.0,2.13,1.0,N,162,90,1.0,8.5,0.5,0.5,1.50,0.00,0.3,13.80,2.5
